# Swaps de Tasa de Interés

## Configuración

### Librerías

In [1]:
from finrisk import QC_Financial_3 as Qcf
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
from datetime import date
import modules.auxiliary as aux

### Variables Globales

Calendario de días feriados.

In [2]:
bus_cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)

Para formato de los `DataFrame`.

In [3]:
format_dict = {
    'nominal': '{0:,.2f}',
    'amort': '{0:,.2f}',
    'interes': '{0:,.2f}',
    'flujo': '{0:,.2f}',
    'amortizacion': '{0:,.2f}',
    'icp_inicial': '{0:,.2f}',
    'icp_final': '{0:,.2f}',
    'valor_tasa': '{0:,.4%}',
    'spread': '{0:,.4%}',
    'gearing': '{0:,.2f}',
    'amort_moneda_pago': '{0:,.2f}',
    'interes_moneda_pago': '{0:,.2f}',
    'valor_indice_fx': '{0:,.2f}',
    'flujo_moneda_nocional': '{0:,.2f}'
}

In [5]:
dicc = {'alvaro': 52, 'diego': 22}

In [6]:
dicc['alvaro']

52

## Definición de un Swap de Tasa de Interés

- Un swap de tasa de interés es un contrato a través del cual dos partes se comprometen a intercambiar flujos de intereses calculados a una tasa fija por intereses calculados a una tasa flotante (algún referencial válido y ampliamente aceptado por el mercado).


- Por ejemplo, cada 3M y por un período de 2Y se intercambian flujos calculados al 0,2181% versus flujos calculados utilizando la Libor USD 3M.
  - Los flujos se deben calcular sobre un cierto nominal (nocional) que nunca se intercambia.

Para más sobre la [Libor](https://www.theice.com/iba/libor), en particular los párrafos bajo el encabezado **The Future of Libor**. 

### Swaps de Libor USD 3M

**Pantalla de cotización en Bloomberg.**

In [9]:
Image(url="img/libor_usd_3m_fixed_2y_swpm.gif", width=900, height=720)

Notar las diferencias entre ambas patas,

- **periodicidad:** en la pata fija (Recibir) la periodicidad es semestral, mientras que en la pata flotante (Pagar) la periodicidad es trimestral.
- **Días (fracción de año):** en la pata fija (Recibir) la fracción de año es 30/360, mientras que en la pata flotante (Pagar) es Act/360.

**Flujos de Caja (la pata flotante es estimada)**

In [10]:
Image(url="img/libor_usd_3m_fixed_2y_cashflows.gif", width=900, height=720)

**¿Cómo construir estos flujos con la librería?**

Partamos con la pata fija.

In [7]:
# Recibo o pago los flujos de esta pata
rp = Qcf.RecPay.RECEIVE

# Fecha de inicio de devengo del primer cupón
fecha_inicio = Qcf.QCDate(17, 9, 2020)

# Fecha final de devengo, antes de ajustes, del último cupón
fecha_final = Qcf.QCDate(17, 9, 2022)

# Regla para ajustes de días feriados
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW

# Periodicidad de pago
periodicidad = Qcf.Tenor('6M')

# Tipo de período irregular (si lo hay)
periodo_irregular = Qcf.StubPeriod.SHORTFRONT

# Número de días después de la fecha final de devengo en que se paga el flujo
lag_pago = 0

# Nocional del contrato
nocional = 10000000.0
amort_es_flujo = True
valor_tasa = .002181
tasa_cupon = Qcf.QCInterestRate(valor_tasa, Qcf.QC30360(), Qcf.QCLinearWf())
moneda = Qcf.QCUSD()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    bus_cal,
    lag_pago,
    nocional,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

In [8]:
type(fixed_rate_leg)

finrisk.QC_Financial_3.Leg

In [10]:
aux.show_leg(fixed_rate_leg, 'FixedRateCashflow').style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-09-17,2021-03-17,2021-03-17,"10,000,000.00",0.00,"10,905.00",True,"10,905.00",USD,0.2181%,Lin30360
1,2021-03-17,2021-09-17,2021-09-17,"10,000,000.00",0.00,"10,905.00",True,"10,905.00",USD,0.2181%,Lin30360
2,2021-09-17,2022-03-17,2022-03-17,"10,000,000.00",0.00,"10,905.00",True,"10,905.00",USD,0.2181%,Lin30360
3,2022-03-17,2022-09-19,2022-09-19,"10,000,000.00","10,000,000.00","11,026.17",True,"10,011,026.17",USD,0.2181%,Lin30360


### Ejercicio

Replicar la tabla anterior y la siguiente en Excel, usando fórmulas no copiando y pegando. (**Hint:** la función DIAS360 en Excel hace el conteo de días en base 30-360).

Pata Flotante

In [9]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.PAY

periodicidad_pago = Qcf.Tenor('3M')
periodo_irregular_pago = Qcf.StubPeriod.SHORTFRONT
lag_pago = 0
periodicidad_fijacion = Qcf.Tenor('3M')
periodo_irregular_fijacion = Qcf.StubPeriod.SHORTFRONT

# vamos a usar el mismo calendario para pago y fijaciones
lag_de_fijacion = 2 # La Libor USD 3M de la fecha f representa una inversión entre f+2d y f+2d+3M

# Definición del índice
codigo = 'LIBORUSD3M'
lin_act360 = Qcf.QCInterestRate(.0024613, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('2d')
tenor = Qcf.Tenor('3m')
usd = Qcf.QCUSD()
libor_usd_3m = Qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    bus_cal,
    bus_cal,
    usd
)
# Fin índice

spread = .0
gearing = 1.0  # intereses -> gearing * Libor + spread

ibor_leg = Qcf.LegFactory.build_bullet_ibor2_leg(
    rp, fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    bus_cal,
    lag_pago,
    periodicidad_fijacion,
    periodo_irregular_fijacion,
    bus_cal,
    lag_de_fijacion,
    libor_usd_3m,
    nocional,
    amort_es_flujo,
    moneda,
    spread,
    gearing)

In [10]:
aux.show_leg(ibor_leg, 'IborCashflow').style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2020-09-17,2020-12-17,2020-09-15,2020-12-17,"-10,000,000.00",0.00,"-6,221.62",True,"-6,221.62",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360
1,2020-12-17,2021-03-17,2020-12-15,2021-03-17,"-10,000,000.00",0.00,"-6,153.25",True,"-6,153.25",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360
2,2021-03-17,2021-06-17,2021-03-15,2021-06-17,"-10,000,000.00",0.00,"-6,289.99",True,"-6,289.99",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360
3,2021-06-17,2021-09-17,2021-06-15,2021-09-17,"-10,000,000.00",0.00,"-6,289.99",True,"-6,289.99",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360
4,2021-09-17,2021-12-17,2021-09-15,2021-12-17,"-10,000,000.00",0.00,"-6,221.62",True,"-6,221.62",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360
5,2021-12-17,2022-03-17,2021-12-15,2022-03-17,"-10,000,000.00",0.00,"-6,153.25",True,"-6,153.25",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360
6,2022-03-17,2022-06-17,2022-03-15,2022-06-17,"-10,000,000.00",0.00,"-6,289.99",True,"-6,289.99",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360
7,2022-06-17,2022-09-19,2022-06-15,2022-09-19,"-10,000,000.00","-10,000,000.00","-6,426.73",True,"-10,006,426.73",USD,LIBORUSD3M,0.2461%,0.0000%,1.00,LinAct360


**¿Para qué sirven?**

- Empresas que tengan deuda en tasa flotante y quieran protegerse contra subidas de tasas de interés.

  - Crédito Libor + s: -(Libor + s)
  - Swap Fija vs Libor: 
    - +(Libor)
    - -(Fija)
  - Costo neto: -(Fija + s)


- Inversionistas con activos a tasa fija y quieran protegerse contra subidas de tasa de interés.
  - Sólo con bonos: si las tasas bajan eres feliz
  - Sólo con bonos: si las tasas suben eres infeliz
  - Si haces swaps: con tasas al alza te puedes cubrir con swap -Fijo y +Libor.


- Empresas con deuda a tasa fija que quieran protegerse contra caídas de las tasas de interés.
  - Empresa no prepaga (se ahorra el costo) y toma un swap en donde +Fija y -Flotante.


- Toma de posiciones direccionales en tasa de interés (al alza y a la baja).
  - De todo lo anterior se deduce que se puede especular en este mercado. Libor es el mercado más líquido.

**Otros tipos de swaps**

- Flotante vs Flotante en la misma divisa: por ejemplo un swap en el cual se intercambian flujos calculados a la Libor USD de 3M por flujos calculados a la Libor USD de 1Y.


- Fijo vs Fijo en dos divisas: por ejemplo un swap en el que se intercambian flujos calculados sobre un nocional de $M_{USD}$ USD usando una tasa fija de $r_{USD}$ por flujos calculados sobre un nocional de $M_{EUR}$ EUR usando una tasa fija de $r_{EUR}$ (los swaps con dos divisas se llaman Cross Currency Swaps (CCS)).


- Fijo vs Flotante en dos divisas: por ejemplo un swap en el que se intercambian flujos calculados sobre un nocional de $M_{USD}$ USD usando una tasa fija de $r_{USD}$ por flujos calculados sobre un nocional de $M_{EUR}$ EUR usando Libor EUR de 3M.


- Flotante vs Flotante en dos divisas: por ejemplo un swap en el que se intercambian flujos calculados sobre un nocional de $M_{USD}$ USD usando una Libor USD de 3M por flujos calculados sobre un nocional de $M_{EUR}$ EUR usando Libor EUR de 3M.

## Overnight Index Swaps (OIS)

- Los OIS son swaps en los cuales se intercambian flujos calculados a una tasa fija por flujos calculados a partir de una tasa overnight interbancaria componiendo intereses.


- Por ejemplo, en USD se utiliza la effective federal funds rate, en EUR el llamado EONIA y en GBP el índice SONIA.


- La effective federal funds rate corresponde al promedio de las transacciones de financiamiento no colateralizado informado a la Reserva Federal de Nueva York (lineal Act/360).


- El índice EONIA (European Overnight Index Average) se computa como el promedio de las transacciones overnight de un grupo pre establecido de bancos (lineal Act/360).


- El índice SONIA (Sterling Overnight Index Average) se calcula como el promedio de las transacciones overnight en Londres ejecutadas a través de WMBA (The Wholesale Markets Brokers’ Association). Este índice es lineal Act/365.


- El mercado USD está transitando hacia el uso de un nuevo benchmark overnight, la Secured Overnight Financing Rate o **SOFR**. Ver [SOFR Primer](https://www.sifma.org/wp-content/uploads/2019/07/SIFMA-Insights-SOFR-Primer.pdf).

### Flujos de un OIS

- En un OIS, los flujos de la pata flotante se obtienen de la composición de los factores de capitalización asociados al índice overnight.


- Por ejemplo, suponiendo que para la pata flotante de un OIS se utiliza la effective fed funds rate de agosto 2014:

In [12]:
# Se importa el archivo Excel a un DataFrame de pandas
eff_1 = pd.read_excel('data/effective_fed_funds_1.xlsx')

# Los campos de fecha se importan como datetime (con horas, minutos y segundos).
# Se mantiene sólo la fecha.
eff_1['fecha_inicial'] = eff_1['fecha_inicial'].dt.date
eff_1['fecha_final'] = eff_1['fecha_final'].dt.date

# Se visualiza el resultado
eff_1.style.format({'effective_fed_funds': '{0:.4%}'})

,fecha_inicial,fecha_final,effective_fed_funds
0,2014-08-01,2014-08-04,0.0900%
1,2014-08-04,2014-08-05,0.0900%
2,2014-08-05,2014-08-06,0.0900%
3,2014-08-06,2014-08-07,0.0900%
4,2014-08-07,2014-08-08,0.0900%
5,2014-08-08,2014-08-11,0.0900%
6,2014-08-11,2014-08-12,0.0900%
7,2014-08-12,2014-08-13,0.0900%
8,2014-08-13,2014-08-14,0.0900%
9,2014-08-14,2014-08-15,0.0900%


Primero se calcula el factor de capitalización entre las fechas iniciales y finales de cada una de las tasas. Se define una pequeña función auxiliar:

In [13]:
def get_wf(fecha_inicial: date, fecha_final: date, valor_tasa: float) -> float:
    """
    Retorna el factor de capitalización para una tasa entre dos fechas.
    Supone que la convención de la tasa es LinAct360.
    """
    qc_fec_ini = Qcf.QCDate(fecha_inicial.day, fecha_inicial.month, fecha_inicial.year)
    
    qc_fec_fin = Qcf.QCDate(fecha_final.day, fecha_final.month, fecha_final.year)
    
    qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
    return qc_tasa.wf(qc_fec_ini, qc_fec_fin)

Se aplica esta función para agregar la columna **wf** al `DataFrame` anterior.

In [15]:
eff_1['wf'] = eff_1.apply(
    lambda row: get_wf(row['fecha_inicial'], row['fecha_final'], row['effective_fed_funds']),
    axis=1
)

Se visualiza el resultado

In [16]:
eff_1.style.format({'effective_fed_funds': '{0:.4%}', 'wf': '{0:.6%}'})

,fecha_inicial,fecha_final,effective_fed_funds,wf
0,2014-08-01,2014-08-04,0.0900%,100.000750%
1,2014-08-04,2014-08-05,0.0900%,100.000250%
2,2014-08-05,2014-08-06,0.0900%,100.000250%
3,2014-08-06,2014-08-07,0.0900%,100.000250%
4,2014-08-07,2014-08-08,0.0900%,100.000250%
5,2014-08-08,2014-08-11,0.0900%,100.000750%
6,2014-08-11,2014-08-12,0.0900%,100.000250%
7,2014-08-12,2014-08-13,0.0900%,100.000250%
8,2014-08-13,2014-08-14,0.0900%,100.000250%
9,2014-08-14,2014-08-15,0.0900%,100.000250%


Se calcula el producto de todos los factores de capitalización y luego se deduce la tasa equivalente en convención LinAct360 entre la primera fecha inicial y la última fecha final.

In [17]:
# Producto de los factores de capitalización
prod = eff_1['wf'].product()

# Fecha inicial en formato QCDate
qc_fec_ini = Qcf.QCDate(
    eff_1.iloc[0]['fecha_inicial'].day,
    eff_1.iloc[0]['fecha_inicial'].month,
    eff_1.iloc[0]['fecha_inicial'].year
)

# Fecha final en formato QCDate
qc_fec_fin = Qcf.QCDate(
    eff_1.iloc[20]['fecha_final'].day,
    eff_1.iloc[20]['fecha_final'].month,
    eff_1.iloc[20]['fecha_final'].year
)

# Número de días entre la fecha inicial y la fecha final
dias = qc_fec_ini.day_diff(qc_fec_fin)

# Tasa equivalente
# prod = 1 + tasa_eq * dias / 360
tasa_eq = (prod - 1) * 360.0 / dias

# Muestra resultados
print(f'Producto wf: {prod:.6%}')
print(f'Días totales: {dias}')
print(f'Tasa equivalente: {tasa_eq:.6%}')

Producto wf: 100.007584%
Días totales: 31
Tasa equivalente: 0.088068%


Finalmente, para obtener los intereses de la pata flotante, se aplica la tasa equivalente, por el número de días del cupón, al nocional del contrato.

In [18]:
nocional = 100000000
interes_flot_ois = nocional * tasa_eq * dias / 360
print(f"Para un nocional de USD {nocional:,.0f} los intereses de la pata flotante del OIS son:\
 USD {interes_flot_ois:,.2f} .")

Para un nocional de USD 100,000,000 los intereses de la pata flotante del OIS son: USD 7,583.60 .


Utilizando la librería:

In [19]:
qc_tasa_eq = Qcf.QCInterestRate(tasa_eq, Qcf.QCAct360(), Qcf.QCLinearWf())
print(f'Intereses: USD {nocional * (qc_tasa_eq.wf(qc_fec_ini, qc_fec_fin) - 1):,.2f}')

Intereses: USD 7,583.60


- Se cotizan distintas madureces (con distinta liquidez) hasta 30Y.


- Los tenors (madureces) de corto plazo son contratos que tienen una única compensación al vencimiento.


- Los tenors desde 2Y en adelante suelen tener cupones intermedios, siendo las periodicidades más comunes 1M, 3M, 6M y 1Y.


- En particular, dado que el índice flotante es overnight, este parámetro es totalmente configurable en un OIS.


- Esto no es así en un swap de Libor u otro índice similar, en donde nos vemos restringidos por el plazo de la tasa flotante.

## Swaps del Mercado Chileno

### Swaps Cámara Promedio

- En el mercado interbancario chileno, uno de los swaps que se opera regularmente y para el cual es posible encontrar precios de mercado a distintos plazos es el **swap cámara promedio en CLP**.


- El comportamiento de la tasa flotante replica el costo de financiamiento (beneficio de inversión) que se tiene tomando (pasando) fondos en forma diaria a la [tasa interbancaria promedio a 1 día](https://www.bcentral.cl/web/banco-central/areas/mercados-financieros) componiendo intereses.


- Dicho de otra manera, un swap cámara promedio en CLP, es un OIS, cuyo índice overnight es la tasa interbancaria promedio a 1 día.

- Los plazos líquidos que los bancos cotizan son 3M, 6M, 9M, 1Y, 1Y6M, plazos anuales entre 2Y y 10Y, 12Y, 15Y, 20Y


- Así como los OIS en otras divisas, los cinco plazos más cortos son swaps sin cupones, es decir hay una única compensación de intereses al vencimiento del swap.


- Los swaps de plazos más largos tienen pagos semestrales en ambas patas.

In [23]:
Image(url="img/20200917_swap_icp.png", width=450*1.2, height=360*1.2)

**Fuente:** Closing de ICAP al 17-09-2020.

Para evitar ambigüedades en el número de decimales a utilizar al momento de calcular el factor de capitalización de la pata flotante, la Chilean Benchmark Facility (CFA) publica diariamente el Índice Cámara Promedio (ICP):

El ICP tiene un valor inicial de 10.000,00 al día 2-9-2002 y se reajusta diariamente según la siguiente fórmula:

$$ICP_{t+d}=Round \left( ICP_{t}\cdot\left(1+r_{t}\cdot \frac{d}{360} \right),2 \right)$$

Donde:

- $r_{t}$: es la tasa interbancaria promedio a la fecha $t$.
- $d$ es el número de días entre $t$ y la próxima fecha hábil.
- $ICP_{t}$ es el valor del $ICP$ en la fecha $t$.
- $ICP_{t+d}$ es el valor del $ICP$ en la fecha hábil siguiente a $t$.
- $Round\left(x,2\right)$ redondea el número $x$ a 2 decimales.

De esta forma la tasa utilizada para calcular el pago de un cupón entre las fechas $T > t$ de la pata flotante de un swap cámara promedio $(TNA)$ se determina a través de:

$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

Donde:

- TNA: significa *tasa nominal anual*.
- $ICP_{\tau}$: es el valor del $ICP$ la fecha $\tau=t,T$.
- $d$ es el número de días entre $t$ y $T$.
- $Round\left(x,4\right)$ redondea el número $x$ a 4 decimales.

#### Flujos de un Swap Cámara Promedio

Veamos la estructura de la pata flotante del swap a 2Y.

In [24]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(21, 9, 2020)
fecha_final = Qcf.QCDate(21, 9, 2022)
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = Qcf.Tenor('6M')
periodo_irregular_pago = Qcf.StubPeriod.SHORTFRONT
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True
spread = .0
gearing = 1.0

icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp_leg(
    rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
    periodo_irregular_pago, bus_cal, lag_pago,
    nominal, amort_es_flujo, spread, gearing)

In [25]:
aux.show_leg(icp_clp_leg, 'IcpClpCashflow').style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-09-21,2021-03-22,2021-03-22,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
1,2021-03-22,2021-09-21,2021-09-21,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
2,2021-09-21,2022-03-21,2022-03-21,"1,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
3,2022-03-21,2022-09-21,2022-09-21,"1,000,000.00","1,000,000.00",True,"1,000,000.00",CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360


Al construir la estructura, a todos los valores de ICP se les asigna el valor por defecto 10,000.00. Veamos como actualizarlo.

In [26]:
tpm = .0005
icp0 = 10000.0

print(f'TPM: {tpm:.2%}')
print(f'ICP Inicial: {icp0:,.2f}')

for i in range(icp_clp_leg.size()):
    cshflw = icp_clp_leg.get_cashflow_at(i)
    icp1 = icp0 * (1 + tpm / 2)
    cshflw.set_start_date_icp(icp0)
    cshflw.set_end_date_icp(icp1)
    icp0 = icp1

TPM: 0.05%
ICP Inicial: 10,000.00


In [27]:
aux.show_leg(icp_clp_leg, 'IcpClpCashflow').style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-09-21,2021-03-22,2021-03-22,"1,000,000.00",0.00,True,252.78,CLP,"10,000.00","10,002.50",0.0500%,252.78,0.0000%,1.00,LinAct360
1,2021-03-22,2021-09-21,2021-09-21,"1,000,000.00",0.00,True,254.17,CLP,"10,002.50","10,005.00",0.0500%,254.17,0.0000%,1.00,LinAct360
2,2021-09-21,2022-03-21,2022-03-21,"1,000,000.00",0.00,True,251.39,CLP,"10,005.00","10,007.50",0.0500%,251.39,0.0000%,1.00,LinAct360
3,2022-03-21,2022-09-21,2022-09-21,"1,000,000.00","1,000,000.00",True,"1,000,255.56",CLP,"10,007.50","10,010.00",0.0500%,255.56,0.0000%,1.00,LinAct360


In [28]:
# Recibo o pago los flujos de esta pata
rp = Qcf.RecPay.PAY

# Fecha de inicio de devengo del primer cupón
fecha_inicio = Qcf.QCDate(21, 9, 2020)

# Fecha final de devengo, antes de ajustes, del último cupón
fecha_final = Qcf.QCDate(21, 9, 2022)

# Regla para ajustes de días feriados
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW

# Periodicidad de pago
periodicidad = Qcf.Tenor('6M')

# Tipo de período irregular (si lo hay)
periodo_irregular = Qcf.StubPeriod.SHORTFRONT

# Número de días después de la fecha final de devengo en que se paga el flujo
lag_pago = 0

# Nocional del contrato
nocional = 1000000.0
amort_es_flujo = True
valor_tasa = .00615
tasa_cupon = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLP()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg_icp = Qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    bus_cal,
    lag_pago,
    nocional,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono)

In [29]:
aux.show_leg(fixed_rate_leg_icp, 'FixedRateCashflow').style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-09-21,2021-03-22,2021-03-22,"-1,000,000.00",0.00,"-3,109.17",True,"-3,109.17",CLP,0.6150%,LinAct360
1,2021-03-22,2021-09-21,2021-09-21,"-1,000,000.00",0.00,"-3,126.25",True,"-3,126.25",CLP,0.6150%,LinAct360
2,2021-09-21,2022-03-21,2022-03-21,"-1,000,000.00",0.00,"-3,092.08",True,"-3,092.08",CLP,0.6150%,LinAct360
3,2022-03-21,2022-09-21,2022-09-21,"-1,000,000.00","-1,000,000.00","-3,143.33",True,"-1,003,143.33",CLP,0.6150%,LinAct360


### Cross Currency Swap UF vs Cámara Promedio

Además del swap cámara promedio, en Chile también se transa un cross currency swap CLFCLP.
Este contrato, tiene una pata en UF a tasa fija y una pata en CLP con la misma estructura que la pata flotante de un swap cámara promedio.

- Para este producto, los plazos líquidos que los bancos cotizan son 3M, 6M, 9M, 1Y, 1Y6M, plazos anuales entre 2Y y 10Y, 12Y, 15Y y 20Y.


- También en este caso, los cinco primeros plazos son swaps sin cupones, es decir hay una única compensación de intereses al vencimiento del swap.


- Adicionalmente, los primeros cinco plazos son con tenor “a la chilena”, es decir, venciendo los días 9 o el día hábil anterior en caso de feriado.


- Estos contratos, además, compensan al vencimiento, los valores de los nocionales en CLP y CLF.

In [30]:
Image(url="img/20200917_icap_xccy_clfclp.png", width=450*1.2, height=360*1.2)

$r_{CLP} = r_{UF} + inf$

0.5 = t_uf + 2.3

**Fuente:** Closing de ICAP al 17-09-2020.

#### Ejercicio

Utilizando la librería (o a mano) construya los flujos del XCCY UF-CLP a 2Y (bid u offer). Utilice la estimación que quiera para los valores del ICP (flujos de la pata flotante) y los valores de la UF (flujos en CLP de la pata fija en CLF).

**Pata Fija** Sólo se muestra esta, la pata ICP ya se mostró en el ejemplo de los swaps cámar promedio.

In [31]:
# Primero se debe dar de alta un FXRateIndex
clf = Qcf.QCCLF()
clp = Qcf.QCCLP()
clfclp = Qcf.FXRate(clf, clp)
zero_d = Qcf.Tenor('0D')
uf = Qcf.FXRateIndex(clfclp, 'UF', zero_d, zero_d, bus_cal)

# Luego se dan de alta los otros parámetros requeridos para la construcción
rp = Qcf.RecPay.RECEIVE
nominal = 300000
fecha_inicio = Qcf.QCDate(22, 9, 2020)
fecha_final = Qcf.QCDate(22, 9, 2022)
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad = Qcf.Tenor('6M')
periodo_irregular = Qcf.StubPeriod.NO
tasa_cupon = Qcf.QCInterestRate(-.0183, Qcf.QCAct360(), Qcf.QCLinearWf())
lag_pago = 0
es_bono = False

# Se da de alta el objeto
fixed_rate_mccy_leg = Qcf.LegFactory.build_bullet_fixed_rate_mccy_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    bus_cal,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    clf,
    clp,
    uf,
    0,
    es_bono
)

In [32]:
tabla = []
for i in range(0, fixed_rate_mccy_leg.size()):
    tabla.append(Qcf.show(fixed_rate_mccy_leg.get_cashflow_at(i)))
for t in tabla:
    print(t)
    break

('2020-09-22', '2021-03-22', '2021-03-22', 300000.0, 0.0, -2760.2499999999973, True, -2760.2499999999973, 'CLF', -0.0183, 'LinAct360', '2021-03-22', 'CLP', 'UF', 1.0, 0.0, -2760.2499999999973)


In [33]:
aux.show_leg(fixed_rate_mccy_leg, 'FixedRateMultiCurrencyCashflow', '').style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_cashflow,flujo_moneda_nocional,moneda_nocional,valor_tasa,tipo_tasa,fecha_fixing_fx,moneda_pago,codigo_indice_fx,valor_indice_fx,amort_moneda_pago,interes_moneda_pago
0,2020-09-22,2021-03-22,2021-03-22,"300,000.00",0.00,"-2,760.25",True,"-2,760.25",CLF,-1.8300%,LinAct360,2021-03-22,CLP,UF,1.00,0.00,"-2,760.25"
1,2021-03-22,2021-09-22,2021-09-22,"300,000.00",0.00,"-2,806.00",True,"-2,806.00",CLF,-1.8300%,LinAct360,2021-09-22,CLP,UF,1.00,0.00,"-2,806.00"
2,2021-09-22,2022-03-22,2022-03-22,"300,000.00",0.00,"-2,760.25",True,"-2,760.25",CLF,-1.8300%,LinAct360,2022-03-22,CLP,UF,1.00,0.00,"-2,760.25"
3,2022-03-22,2022-09-22,2022-09-22,"300,000.00","300,000.00","-2,806.00",True,"297,194.00",CLF,-1.8300%,LinAct360,2022-09-22,CLP,UF,1.00,"300,000.00","-2,806.00"


In [35]:
inflacion = .02
uf = 30000
for i in range(fixed_rate_mccy_leg.size()):
    f = fixed_rate_mccy_leg.get_cashflow_at(i)
    uf1 = uf0 * (1 + inflacion)**(.5)
    f.set_fx_rate_index_value(uf1)
    uf0 = uf1

NameError: name 'uf0' is not defined

In [ ]:
aux.show_leg(fixed_rate_mccy_leg, 'FixedRateMultiCurrencyCashflow', '').style.format(format_dict)